In [1]:
import geopandas as gpd
import pandas as pd
import os, datetime

In [2]:
path = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"

In [3]:
# 2 years * 3 AOI * 2 travel modes * 2 services
sas = ["Jobs", "Amenities"]
travel_modes = ["Biking", "Walking"]
layer_names = ["baseyearJobs_FeatureToPoint", "forecastJobs_FeatureToPoint"]
AOIs = ["MPO", "EFA", "NEFA"]
# matched with layer names
jobfields = ["ojobs", "jobs"]
years = [2020, 2045]
hhfields = ["ohh", "hh"]

In [4]:
import JoinHHTables as jht

In [12]:
def calculateAccessibility(service = "Jobs",
                          travel_mode = "Biking",
                          year = 2020,
                          AOI = "MPO",
                          HH = False):
    if HH:
        shp = AOI + service + travel_mode + str(year) + "HH_SA"
    else:
        shp = AOI + service + travel_mode + str(year)
    
    if HH:
        access = jht.UpateHHTables(AOI = AOI, service = service, year = year, travel_mode = travel_mode)
    else:
        access = gpd.read_file(path, layer = shp)
    
    if service == "Jobs":
        if HH:
            targetfield = jobfields[years.index(year)]
        else:
            targetfield = layer_names[years.index(year)] + "_" + jobfields[years.index(year)]
        newfield = 'weighted_' + jobfields[years.index(year)]
    else:
        targetfield = "Join_Count"
        newfield = 'weighted_count'
    
    hhfield = hhfields[years.index(year)]
    access[newfield] = access[targetfield] * access[hhfield]
    
    return round(access[newfield].sum()/access[hhfield].sum())

In [15]:
outpath = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis"

In [6]:
now = datetime.datetime.now()
for sa in sas:
    byService = []
    for travel_mode in travel_modes:
        byTravelMode = []
        colnms = []
        for year in years:
            byYear = []
            colnm = []
            for AOI in AOIs:
                acc = calculateAccessibility(service = sa,
                                             travel_mode = travel_mode,
                                             year = year,
                                             AOI = AOI)
                print("Got the accessibility number for {0} in {1} by {2} in {3}...".format(sa, AOI, travel_mode, year))
                byYear.append(acc)
                colnm.append(AOI+str(year))
            byTravelMode += byYear
            colnms += colnm
        byService.append(byTravelMode)
    print("Got the accessibility table for " + sa + ":")
    df = pd.DataFrame(byService)
    df.columns = colnms
    df.index= travel_modes
    print(df)
    df.to_csv(os.path.join(outpath, sa+"Access.csv"))
later = datetime.datetime.now()
elapsed = later - now
print("total time used: {0}".format(elapsed))

Got the accessibility number for Jobs in MPO by Biking in 2020...
Got the accessibility number for Jobs in EFA by Biking in 2020...
Got the accessibility number for Jobs in NEFA by Biking in 2020...
Got the accessibility number for Jobs in MPO by Biking in 2045...
Got the accessibility number for Jobs in EFA by Biking in 2045...
Got the accessibility number for Jobs in NEFA by Biking in 2045...
Got the accessibility number for Jobs in MPO by Walking in 2020...
Got the accessibility number for Jobs in EFA by Walking in 2020...
Got the accessibility number for Jobs in NEFA by Walking in 2020...
Got the accessibility number for Jobs in MPO by Walking in 2045...
Got the accessibility number for Jobs in EFA by Walking in 2045...
Got the accessibility number for Jobs in NEFA by Walking in 2045...
Got the accessibility table for Jobs:
         MPO2020  EFA2020  NEFA2020  MPO2045  EFA2045  NEFA2045
Biking     36253    42490     34634    50065    61479     46912
Walking     2513     3761      2

In [16]:
now = datetime.datetime.now()
for sa in sas:
    byService = []
    for travel_mode in travel_modes:
        byTravelMode = []
        colnms = []
        for year in years:
            byYear = []
            colnm = []
            for AOI in AOIs:
                acc = calculateAccessibility(service = sa,
                                             travel_mode = travel_mode,
                                             year = year,
                                             AOI = AOI,
                                             HH = True)
                print("Got the accessibility number for {0} in {1} by {2} in {3}...".format(sa, AOI, travel_mode, year))
                byYear.append(acc)
                colnm.append(AOI+str(year))
            byTravelMode += byYear
            colnms += colnm
        byService.append(byTravelMode)
    print("Got the accessibility table for " + sa + ":")
    df = pd.DataFrame(byService)
    df.columns = colnms
    df.index= travel_modes
    print(df)
    df.to_csv(os.path.join(outpath, sa+"AccessHH.csv"))
later = datetime.datetime.now()
elapsed = later - now
print("total time used: {0}".format(elapsed))

Got the accessibility number for Jobs in MPO by Biking in 2020...
Got the accessibility number for Jobs in EFA by Biking in 2020...
Got the accessibility number for Jobs in NEFA by Biking in 2020...
Got the accessibility number for Jobs in MPO by Biking in 2045...
Got the accessibility number for Jobs in EFA by Biking in 2045...
Got the accessibility number for Jobs in NEFA by Biking in 2045...
Got the accessibility number for Jobs in MPO by Walking in 2020...
Got the accessibility number for Jobs in EFA by Walking in 2020...
Got the accessibility number for Jobs in NEFA by Walking in 2020...
Got the accessibility number for Jobs in MPO by Walking in 2045...
Got the accessibility number for Jobs in EFA by Walking in 2045...
Got the accessibility number for Jobs in NEFA by Walking in 2045...
Got the accessibility table for Jobs:
         MPO2020  EFA2020  NEFA2020  MPO2045  EFA2045  NEFA2045
Biking     34020    16566     17949    44976    20514     25156
Walking     2323     2003      1

In [5]:
EquityAreaID = pd.read_csv("../EquityAreaID.csv")

In [9]:
def GetEFA_numbers_HH(service = "Jobs", travel_mode = 'Biking', year = 2020):
    AOI = "EFA"
    byYear = []
    colnm = []
    for i in EquityAreaID.index:
        EFA_ID = EquityAreaID['EquityArea'].values[i]
        access = jht.UpateHHTables(AOI = AOI, service = service, year = year, travel_mode = travel_mode, 
                  EFA_ID = EFA_ID, EFA = True)
        if service == "Jobs":
            targetfield = jobfields[years.index(year)]
            newfield = 'weighted_' + jobfields[years.index(year)]
        else:
            targetfield = "Join_Count"
            newfield = 'weighted_count'

        hhfield = hhfields[years.index(year)]
        access[newfield] = access[targetfield] * access[hhfield]
        
        acc = round(access[newfield].sum()/access[hhfield].sum())

        print("Got the accessibility number for {0} in {1} by {2} in {3}...".format(service, AOI + str(EFA_ID), travel_mode, year))
        byYear.append(acc)
        colnm.append(AOI + str(EFA_ID) + "_" + str(year))
    return byYear, colnm

In [10]:
now = datetime.datetime.now()
for sa in sas:
    byService = []
    for travel_mode in travel_modes:
        byTravelMode = []
        colnms = []
        for year in years:
            out = GetEFA_numbers_HH(service = sa,
                                  travel_mode = travel_mode,
                                  year = year)
            byYear = out[0]
            colnm = out[1]
            byTravelMode += byYear
            colnms += colnm
        byService.append(byTravelMode)
    print("Got the accessibility table for " + sa + ":")
    df = pd.DataFrame(byService)
    df.columns = colnms
    df.index= travel_modes
    print(df)
    df.to_csv(os.path.join(outpath, sa + "AccessEFA_HH.csv"))
later = datetime.datetime.now()
elapsed = later - now
print("total time used: {0}".format(elapsed))

Got the accessibility number for Jobs in EFA5 by Biking in 2020...
Got the accessibility number for Jobs in EFA6 by Biking in 2020...
Got the accessibility number for Jobs in EFA10 by Biking in 2020...
Got the accessibility number for Jobs in EFA18 by Biking in 2020...
Got the accessibility number for Jobs in EFA29 by Biking in 2020...
Got the accessibility number for Jobs in EFA31 by Biking in 2020...


<ipython-input-9-c291925d070e>:19: RuntimeWarning: invalid value encountered in double_scalars
  acc = round(access[newfield].sum()/access[hhfield].sum())


ValueError: cannot convert float NaN to integer